In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function

from collections import defaultdict

from intervaltree import IntervalTree, Interval

In [3]:
def overlap_intervals(self, other):
    splits = (self | other)
    splits.split_overlaps()
    self_int_other = IntervalTree(filter(lambda r: self.overlaps(r) and other.overlaps(r), splits))
    return self_int_other


def overlap_all(trees):
    if len(trees) < 2:
        return None

    overlap = overlap_intervals(trees[0], trees[1])
    for t in trees[2:]:
        overlap = overlap_intervals(overlap, t)
    overlap.merge_overlaps(data_reducer)

    return overlap


def data_reducer(current_data, new_data):
    return current_data | new_data

In [4]:
gtrees = defaultdict(dict)
all_genes = set()
files = defaultdict(lambda: defaultdict(list))
#allref= defaultdict(lambda: defaultdict(list))

In [5]:
for f in ('vqc', 'iqc', 'sqc', 'mqc'):
    for line in open(f): 
            x = line.rstrip().split(' ')
            if  "Real" in  str(x[0]):
                files[f][x[14]].append((int(x[3]), int(x[4])))
print('done')

done


In [6]:
gtrees = defaultdict(dict)
all_genes = set()
for f in files:
    for g in files[f]:
        all_genes.add(g)
        gtrees[g][f] = IntervalTree(Interval(iv[0], iv[1], set([f]))
                                    for iv in files[f][g])
print('done')

done


In [7]:
genome_size = 2056037
overlap_trees = {}
unaligned_regions = {}
for gene in all_genes:
   # overlap_trees[gene] = overlap_all(list(gtrees[gene].values()))
    unaligned = IntervalTree()
    unaligned[1:genome_size] = set()

    if gtrees[gene]:
        for f in gtrees[gene]:
            for interval in gtrees[gene][f]:
                unaligned.chop(interval.begin, interval.end)
    
    unaligned_regions[gene] = unaligned

In [8]:
overlap_unaligned = overlap_all(list(unaligned_regions.values()))

In [9]:
for interval in sorted(overlap_unaligned):
    print(interval.begin, interval.end)

In [10]:
print(overlap_unaligned['Deinococcus_radiodurans_R1_chromosome_1__complete_sequence'])
#print()
print(unaligned_regions['Deinococcus_radiodurans_R1_chromosome_1__complete_sequence'])

set([])
IntervalTree([Interval(7342, 19915, set([])), Interval(20410, 21101, set([])), Interval(22108, 51023, set([])), Interval(54386, 56369, set([])), Interval(64308, 81630, set([])), Interval(81960, 113510, set([])), Interval(114726, 138190, set([])), Interval(139946, 201166, set([])), Interval(209767, 218790, set([])), Interval(231983, 249747, set([])), Interval(250349, 254037, set([])), Interval(269054, 276619, set([])), Interval(277336, 309679, set([])), Interval(312361, 321162, set([])), Interval(322193, 363778, set([])), Interval(368585, 444551, set([])), Interval(445866, 445868, set([])), Interval(447359, 481256, set([])), Interval(490460, 490647, set([])), Interval(502299, 502300, set([])), Interval(506337, 582329, set([])), Interval(583308, 670584, set([])), Interval(674969, 766458, set([])), Interval(769065, 769091, set([])), Interval(791611, 804060, set([])), Interval(814613, 823887, set([])), Interval(832597, 871293, set([])), Interval(871848, 871856, set([])), Interval(8

## End of modifications

In [11]:
cutoff =0
out =open('unaligned-regions','w+')
for gene in all_genes:
        if len(overlap_unaligned[gene]) >=cutoff:
                for i in overlap_unaligned[gene]:
                        print(gene,',',i,',',file=out)
                        print(gene,',',i,',')
print ('done ')                


done 


In [15]:
#parse case files
gdict=defaultdict(list)
input=open('unaligned-regions','r')
for line in input:
        line=line.split(',')
        #print(line)
        gene=line[0]
        x=line[1].split('(')
        x=x[1]
        y=line[2]
        #print(x,"-",y)
        gdict[line[0]].append(x)
        gdict[line[0]].append(y)


for g in gdict.keys(): 
         i=1
         l=len(gdict[g])
         min =20700000
         max =-1
         sum=0
         while(i <l-1) :
                gap=abs( int(gdict[g][i+1]) -int(gdict[g][i]) )
                sum+=gap
                if min >gap: 
                        min=gap
                if max <gap:
                        max=gap
                i=i+2
         print(g,"has ",l/2,"unaligned with min/max/average gaps:",str(min), "/",str(max),"/",(sum/l))

In [13]:
for i in overlap_unaligned['Desulfovibrio_vulgaris_DP4']:
      print(i)

In [14]:
#t = IntervalTree([Interval(2, 5),Interval(7,9)])
t=IntervalTree([Interval(0,10)])
t.chop(3,7)
#t.split_overlaps()
sorted(t)

[Interval(0, 3), Interval(7, 10)]